In [16]:
import torchaudio
from datasets import load_dataset, load_metric
from transformers import (
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
)
import torch
import re
import sys

model_name = "facebook/wav2vec2-large-xlsr-53-portuguese"
device = "cuda"
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'  # noqa: W605

In [17]:
model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)
processor = Wav2Vec2Processor.from_pretrained(model_name)

In [18]:
# opening a single file to test
from scipy.io import wavfile
samplerate, data = wavfile.read('../raw_videos/Em10_Fala1_CarolinaHolly/audio/Em10_Fala1_CarolinaHolly.wav')

In [19]:
features = processor(data, sampling_rate=samplerate, padding=True, return_tensors="pt")
input_values = features.input_values.to(device)
attention_mask = features.attention_mask.to(device)
with torch.no_grad():
    logits = model(input_values, attention_mask=attention_mask).logits
pred_ids = torch.argmax(logits, dim=-1)
predicted = processor.batch_decode(pred_ids)


In [20]:
predicted

['filha quando olhe para você e vejá essa fagula de vida que brilha nos seus olhos juro que me sinto agraciada por um milagre nem meus sonhos eu poderia imaginar que você seria tão linda e nem que eu me apaixonaria tão perdidamente por um ser tão pequenino']

In [21]:
pred_ids.shape

torch.Size([1, 1210])

In [23]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from datasets import load_dataset
import soundfile as sf
import torch

# load model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# define function to read in sound file
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

# tokenize
input_values = tokenizer(data, return_tensors="pt", padding="longest").input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)

C:\Users\Filipe\.conda\envs\wav2kp\lib\site-packages\transformers\models\wav2vec2\tokenization_wav2vec2.py:358: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
transcription

["PHILEA FON DO I DI PRAVO SEY VIJO SAFA GUDEGI VIDAKI BRILLIAN OSOZORDIOS JURUKI MESINTAGRACIADA PORUMILMAGRI NAME YOU SONOZUPO THE DAMAGINACI WAS A CIDIA DON'T LINGER IN NAME CAME YA PASSIONED IDEA DON'T PERIJUDAM ANCE BRU SEI DON'T PEKIN INN"]